In [5]:
from mesa import Agent, Model

In [6]:
# Sequences as events
#sequence_events = {
    #"Standard Participatory Process (1)": ["Policy proposal", "Public consultation", "Policy revision", "Final decision"],
    #"Standard Process (2)": ["Policy proposal", "Public consultation", "Final decision"],
    #"Early Participatory Process (3)": ["Public consultation", "Policy proposal", "Public consultation", "Policy revision", "Final decision"],
    #"Private Sector Initiative Process (4)": ["Private sector proposal", "Government review", "Policy proposal", "Public consultation", "Policy revision", "Final decision"]
#}

# Events influence justice characteristics
#event_impact = {
    #"Public consultation": {"Transparency": 0.1, "Inclusivity": 0.1},
    #"Policy proposal": {"Accountability": 0.05},
    #"Policy revision": {"Accountability": 0.1, "Outcome Fairness": 0.1},
    #"Final decision": {"Accountability": 0.05},
    #"Private sector proposal": {"Transparency": -0.1},
    #"Government review": {"Transparency": 0.1, "Accountability": 0.1}
#}

# Expectations per agent

agent_expectations = {
    "Residents (Positive)": {"Transparency": 0.5, "Inclusivity": 0.5, "Accountability": 0.3, "Outcome Fairness": 0.5},
    "Residents (Neutral)": {"Transparency": 0.6, "Inclusivity": 0.6, "Accountability": 0.4, "Outcome Fairness": 0.4},
    "Residents (Negative)": {"Transparency": 0.8, "Inclusivity": 0.8, "Accountability": 0.7, "Outcome Fairness": 0.3},
    "NGOs": {"Transparency": 0.8, "Inclusivity": 0.8, "Accountability": 0.6, "Outcome Fairness": 0.3},
    "Marginalized Groups": {"Transparency": 0.7, "Inclusivity": 0.9, "Accountability": 0.4, "Outcome Fairness": 0.4}
}

# sequences

sequence_events = {
    "Maximal Effort Process": [
        "Early public consultation",
        "Excursion",
        "Consultation: technology",
        "Consultation: nuisance and health",
        "Consultation: ecological impacts",
        "Consultation: noise and safety",
        "Consultation: commercial developer",
        "Consultation: local ownership",
        "Consultation: financial participation",
        "Consultation: external safety",
        "Consultation: Board authority",
        "Policy proposal",
        "Policy revision",
        "Public consultation",
        "Final decision"
    ],
    "Moderate Effort Process": [
        "Early public consultation",
        "Excursion",
        "Consultation: nuisance and health",
        "Consultation: financial participation",
        "Consultation: Board authority",
        "Policy proposal",
        "Public consultation",
        "Policy revision",
        "Final decision"
    ],
    "Basic Effort Process": [
        "Excursion",
        "Consultation: technology",
        "Policy proposal",
        "Public consultation",
        "Final decision"
    ],
    "Minimal Effort Process": [
        "Policy proposal",
        "Final decision"
    ]
}


# expectations per agent
event_impact = {
    "Early public consultation": {"Transparency": 1, "Inclusivity": 1},
    "Excursion": {"Transparency": 0.8, "Inclusivity": 0.6},
    "Consultation: technology": {"Transparency": 0.9, "Inclusivity": 0.7},
    "Consultation: nuisance and health": {"Transparency": 0.8, "Inclusivity": 0.9},
    "Consultation: ecological impacts": {"Transparency": 0.9, "Inclusivity": 0.8},
    "Consultation: noise and safety": {"Transparency": 0.9, "Inclusivity": 0.8},
    "Consultation: commercial developer": {"Transparency": 0.7, "Inclusivity": 0.6},
    "Consultation: local ownership": {"Inclusivity": 1, "Outcome Fairness": 0.8},
    "Consultation: financial participation": {"Outcome Fairness": 1, "Inclusivity": 0.8},
    "Consultation: external safety": {"Transparency": 0.9, "Accountability": 0.6},
    "Consultation: Board authority": {"Accountability": 0.8, "Transparency": 0.7},
    "Policy proposal": {"Accountability": 0.6},
    "Policy revision": {"Accountability": 1, "Outcome Fairness": 1},
    "Public consultation": {"Transparency": 1, "Inclusivity": 1},
    "Final decision": {"Outcome Fairness": 0.7},
}


In [7]:
# stakholder agent class

class StakeholderAgent(Agent):
    def __init__(self, unique_id, model, stakeholder_type):
        super().__init__(model)
        self.unique_id = unique_id
        self.stakeholder_type = stakeholder_type
        self.perceptions = {}
        self.reactions = {}

    def calculate_perceptions(self):
        expectations = agent_expectations[self.stakeholder_type]

        for sequence, events in self.model.sequence_events.items():
            for sequence, events in self.model.sequence_events.items():
                actual_scores = {"Transparency": 0, "Inclusivity": 0, "Accountability": 0, "Outcome Fairness": 0}
                counts = {"Transparency": 0, "Inclusivity": 0, "Accountability": 0, "Outcome Fairness": 0}

                for event in events:
                    impacts = self.model.event_impact.get(event, {})
                    for key in actual_scores:
                        if key in impacts:
                            actual_scores[key] += impacts[key]
                            counts[key] += 1

            # If missing public consultation before policy proposal

                if "Policy proposal" in events:
                    index = events.index("Policy proposal")
                    if "Public consultation" not in events[:index] and "Early public consultation" not in events[:index]:
                        actual_scores["Transparency"] -= 0.5
                        actual_scores["Inclusivity"] -= 0.5
                        counts["Transparency"] += 1
                        counts["Inclusivity"] += 1

                # If no policy revision

                if "Policy revision" not in events:
                    actual_scores["Accountability"] -= 0.5
                    actual_scores["Outcome Fairness"] -= 0.5
                    counts["Accountability"] += 1
                    counts["Outcome Fairness"] += 1

                # Average the scores

                for key in actual_scores:
                    if counts[key] > 0:
                        actual_scores[key] = max(0, actual_scores[key] / counts[key])
                    else:
                        actual_scores[key] = 0

                # Compute squared distance to expectations

                diff = sum((actual_scores[key] - expectations[key]) ** 2 for key in expectations)
                justice_score = 1 - diff
                justice_score = max(0, min(justice_score, 1))

                self.perceptions[sequence] = round(justice_score, 3)
                if justice_score >= 0.7:
                    self.reactions[sequence] = "Support"
                elif justice_score >= 0.4:
                    self.reactions[sequence] = "Neutral"
                else:
                    self.reactions[sequence] = "Oppose"

    # model class
    
    class JusticeModel(Model):
        def __init__(self):
            super().__init__()
            self.stakeholders = []
            self.sequence_events = sequence_events
            self.event_impact = event_impact

            agent_types = list(agent_expectations.keys())
            for i, name in enumerate(agent_types):
                agent = StakeholderAgent(i, self, name)
                self.stakeholders.append(agent)

        def step(self):
            for agent in self.stakeholders:
                agent.calculate_perceptions()


In [8]:
# run model
model = JusticeModel()
model.step()

# output
for agent in model.stakeholders:
    print(f"\n{agent.stakeholder_type}")
    for sequence in sequence_events:
        score = agent.perceptions[sequence]
        reaction = agent.reactions[sequence]
        print(f"  {sequence}: score = {score}, reaction = ({reaction})")


Residents (Positive)
  Maximal Effort Process: score = 0.425, reaction = (Neutral)
  Moderate Effort Process: score = 0.331, reaction = (Oppose)
  Basic Effort Process: score = 0.772, reaction = (Support)
  Minimal Effort Process: score = 0.277, reaction = (Oppose)

Residents (Neutral)
  Maximal Effort Process: score = 0.536, reaction = (Neutral)
  Moderate Effort Process: score = 0.455, reaction = (Neutral)
  Basic Effort Process: score = 0.762, reaction = (Support)
  Minimal Effort Process: score = 0.068, reaction = (Oppose)

Residents (Negative)
  Maximal Effort Process: score = 0.663, reaction = (Neutral)
  Moderate Effort Process: score = 0.623, reaction = (Neutral)
  Basic Effort Process: score = 0.353, reaction = (Oppose)
  Minimal Effort Process: score = 0, reaction = (Oppose)

NGOs
  Maximal Effort Process: score = 0.643, reaction = (Neutral)
  Moderate Effort Process: score = 0.593, reaction = (Neutral)
  Basic Effort Process: score = 0.472, reaction = (Neutral)
  Minimal Ef